# Resnet50 + autoencoder로 각각 이미지와 시계열 셋을 전처리한 후 XGB로 파인튠하여 최종분류
*실제데이터로 트라이

*해결과제
1. 10번쨰열 결측치 처리
2. 데이터 중복시간있는것 처리 -> 10번째열 있는 자료만 쓰려고 하였으나 10번쨰열이 없는 데이터도 상당히 존재함
3. 1~9 -> 10 하는 간단한 모델 적합 후 결측치 처리하는 방향으로 고려중

In [1]:
#import
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json 
import random
from xgboost import XGBClassifier
from tensorflow.keras.applications.resnet import ResNet50
import tensorflow as tf
from tensorflow import keras
from sklearn import metrics
from torch.utils.data import Dataset
import torch


In [2]:
path = os.getcwd()

In [3]:
# 제공된 sample data는 파프리카와 시설포도 2종류의 작물만 존재
label_description = {
 '3_00_0': '파프리카_정상',
 '3_a9_1': '파프리카흰가루병_초기',
 '3_a9_2': '파프리카흰가루병_중기',
 '3_a9_3': '파프리카흰가루병_말기',
 '3_a10_1': '파프리카잘록병_초기',
 '3_a10_2': '파프리카잘록병_중기',
 '3_a10_3': '파프리카잘록병_말기',
 '3_b3_1': '칼슘결핍_초기',
 '3_b3_2': '칼슘결핍_중기',
 '3_b3_3': '칼슘결핍_말기',
 '3_b6_1': '다량원소결핍 (N)_초기',
 '3_b6_2': '다량원소결핍 (N)_중기',
 '3_b6_3': '다량원소결핍 (N)_말기',
 '3_b7_1': '다량원소결핍 (P)_초기',
 '3_b7_2': '다량원소결핍 (P)_중기',
 '3_b7_3': '다량원소결핍 (P)_말기',
 '3_b8_1': '다량원소결핍 (K)_초기',
 '3_b8_2': '다량원소결핍 (K)_중기',
 '3_b8_3': '다량원소결핍 (K)_말기',
 '6_00_0': '시설포도_정상',
 '6_a11_1': '시설포도탄저병_초기',
 '6_a11_2': '시설포도탄저병_중기',
 '6_a11_3': '시설포도탄저병_말기',
 '6_a12_1': '시설포도노균병_초기',
 '6_a12_2': '시설포도노균병_중기',
 '6_a12_3': '시설포도노균병_말기',
 '6_b4_1': '일소피해_초기',
 '6_b4_2': '일소피해_중기',
 '6_b4_3': '일소피해_말기',
 '6_b5_1': '축과병_초기',
 '6_b5_2': '축과병_중기',
 '6_b5_3': '축과병_말기',
}

label_encoder = {key:idx for idx, key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}

In [4]:
# 분석에 사용할 feature 선택
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

csv_files = sorted(glob(path + '\\data\\train\\*\\*.csv'))

temp_csv = pd.read_csv(csv_files[0])[csv_features]
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# feature 별 최대값, 최솟값 계산
for csv in tqdm(csv_files[1:]):
    temp_csv = pd.read_csv(csv)[csv_features]
    temp_csv = temp_csv.replace('-',np.nan).dropna()#.astype(float)
    if len(temp_csv) == 0:
        continue
    temp_csv = temp_csv.astype(float)
    temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
    max_arr = np.max([max_arr,temp_max], axis=0)
    min_arr = np.min([min_arr,temp_min], axis=0)

# feature 별 최대값, 최솟값 dictionary 생성
csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}
csv_feature_dict

100%|██████████| 5759/5759 [00:57<00:00, 100.33it/s]


{'내부 온도 1 평균': [3.4, 47.3],
 '내부 온도 1 최고': [3.4, 47.6],
 '내부 온도 1 최저': [3.3, 47.0],
 '내부 습도 1 평균': [23.7, 100.0],
 '내부 습도 1 최고': [25.9, 100.0],
 '내부 습도 1 최저': [0.0, 100.0],
 '내부 이슬점 평균': [0.1, 34.5],
 '내부 이슬점 최고': [0.2, 34.7],
 '내부 이슬점 최저': [0.0, 34.4]}

In [5]:
# 변수 설명 csv 파일 참조
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}

label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

label_encoder = {key:idx for idx, key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}

In [6]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train'):
        self.mode = mode
        self.files = files
        self.csv_feature_dict = csv_feature_dict
        self.csv_feature_check = [0]*len(self.files)
        self.csv_features = [None]*len(self.files)
        self.max_len = 24 * 6
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('\\')[-1]
        
        # csv
        if self.csv_feature_check[i] == 0:
            csv_path = f'{file}\\{file_name}.csv'
            # csv_path = f'{file_name}.csv'
            df = pd.read_csv(csv_path)[self.csv_feature_dict.keys()]
            df = df.replace('-', 0)
            # MinMax scaling
            for col in df.columns:
                df[col] = df[col].astype(float) - self.csv_feature_dict[col][0]
                df[col] = df[col] / (self.csv_feature_dict[col][1]-self.csv_feature_dict[col][0])
            # zero padding
            pad = np.zeros((self.max_len, len(df.columns)))
            length = min(self.max_len, len(df))
            pad[-length:] = df.to_numpy()[-length:]
            # transpose to sequential data
            csv_feature = pad
            self.csv_features[i] = csv_feature
            self.csv_feature_check[i] = 1
        else:
            csv_feature = self.csv_features[i]
        
        # image
        image_path = f'{file}\\{file_name}.jpg'
        img = cv2.imread(image_path)
        img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32)/255
        # img = np.transpose(img, (2,0,1))
        
        if self.mode == 'train':
            json_path = f'{file}\\{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = f'{crop}_{disease}_{risk}'
            
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32),
                'label' : torch.tensor(self.label_encoder[label], dtype=torch.long)
            }
        else:
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32)
            }

In [7]:
# batch_size = 256
# class_n = len(label_encoder)
# learning_rate = 1e-4
# embedding_dim = 512

num_features = len(csv_feature_dict)
max_len = 144
# dropout_rate = 0.1
# epochs = 10
# vision_pretrain = True

In [34]:
train = sorted(glob(path + '\\data\\train\\*'))[:100]
test = sorted(glob(path + '\\data\\test\\*'))

try:
    train.remove('d:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\data\\train\\20643')
except:
    pass

labelsss = pd.read_csv('data\\train.csv')['label']
# train, val = train_test_split(train, test_size=0.2, stratify=labelsss)

In [35]:
train_dataset = CustomDataset(train)
# val_dataset = CustomDataset(val)
test_dataset = CustomDataset(test, mode = 'test')

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=False)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=16, shuffle=False)

In [36]:
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
# # val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=False)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=16, shuffle=False)

train_dataloader = torch.utils.data.DataLoader(train_dataset ,shuffle=False)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset,shuffle=False)

In [37]:
# csvarr = np.empty((0,144,9), float)
# imgarr = np.empty((0,224,224,3), float)
# lablearr = np.array([])
                 
# for batch, batch_item in enumerate(train_dataloader):
#     # if batch == 1:
#     #     pass
#     # else:
#     #     break
#     csvarr = np.append(csvarr,batch_item['csv_feature'], axis = 0)
#     imgarr = np.append(imgarr,batch_item['img'], axis = 0)
#     lablearr = np.append(lablearr,batch_item['label'])
        
       
                
# csvarr_test = np.empty((0,144,9), float)
# imgarr_test = np.empty((0,224,224,3), float)
# # lablearr_test = np.array([])
                 
# for batch, batch_item in enumerate(test_dataloader):
#     # if batch == 1:
#     #     pass
#     # else:
#     #     break
#     csvarr_test = np.append(csvarr_test,batch_item['csv_feature'], axis = 0)
#     imgarr_test = np.append(imgarr_test,batch_item['img'], axis = 0)
#     # lablearr_test = np.append(lablearr_test,batch_item['label'])
        
        

In [40]:
csvarr = np.empty((0,144,9), float)
imgarr = np.empty((0,224,224,3), float)
lablearr = np.array([])

unit = 500

for i in range(12):
    i = i+1
    train = sorted(glob(path + '\\data\\train\\*'))[(i-1)*unit:i*unit]
    test = sorted(glob(path + '\\data\\test\\*'))

    try:
        train.remove('d:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\data\\train\\20643')
    except:
        pass

    labelsss = pd.read_csv('data\\train.csv')['label']
    # train, val = train_test_split(train, test_size=0.2, stratify=labelsss)
    train_dataset = CustomDataset(train)
    # val_dataset = CustomDataset(val)
    test_dataset = CustomDataset(test, mode = 'test')

    # train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
    # val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=False)
    # test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=16, shuffle=False)
    # train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
    # # val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=False)
    # test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=16, shuffle=False)

    train_dataloader = torch.utils.data.DataLoader(train_dataset ,shuffle=False)
    # val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=False)
    test_dataloader = torch.utils.data.DataLoader(test_dataset,shuffle=False)
    

                 
    for batch, batch_item in enumerate(train_dataloader):
        # if batch == 1:
        #     pass
        # else:
        #     break
        csvarr = np.append(csvarr,batch_item['csv_feature'], axis = 0)
        imgarr = np.append(imgarr,batch_item['img'], axis = 0)
        lablearr = np.append(lablearr,batch_item['label'])

OSError: [Errno 22] Invalid argument: 'd:\\OneDrive - UOS\\allrepos\\dacon_diagnosis\\data\\train\\66997\\66997.csv'

In [ ]:
csvarr_test = np.empty((0,144,9), float)
imgarr_test = np.empty((0,224,224,3), float)
# lablearr_test = np.array([])
                 
for batch, batch_item in enumerate(test_dataloader):
    # if batch == 1:
    #     pass
    # else:
    #     break
    csvarr_test = np.append(csvarr_test,batch_item['csv_feature'], axis = 0)
    imgarr_test = np.append(imgarr_test,batch_item['img'], axis = 0)
    # lablearr_test = np.append(lablearr_test,batch_item['label'])
        

In [ ]:
x_train = [csvarr,imgarr]
y_train = lablearr

x_test = [csvarr_test,imgarr_test]

In [ ]:


class Eencoder(keras.layers.Layer):
    global max_len
    def __init__(self):
        super(Eencoder, self).__init__()
        self.block1_layer1 = tf.keras.layers.Conv1D(9, 24*2, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer2 = tf.keras.layers.Conv1D(18, 24*2+1, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer3 = tf.keras.layers.Conv1D(36, 24*2+1, activation='relu',)#input_shape=input_shape[1:])
         
    def call(self, inputs):
        #LSTM파트
        lstm_x = self.block1_layer1(inputs)
        lstm_x = tf.nn.relu(lstm_x)
        lstm_x = self.block1_layer2(lstm_x)
        lstm_x = tf.nn.relu(lstm_x)
        lstm_x = self.block1_layer3(lstm_x)
        lstm_x = tf.nn.relu(lstm_x)
        
        return lstm_x
class Ddecoder(keras.layers.Layer):
    def __init__(self):
        super(Ddecoder, self).__init__()
        self.block1_layer1 = tf.keras.layers.Conv1DTranspose(3,  24*2, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer2 = tf.keras.layers.Conv1DTranspose(6,  24*2+1, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer3 = tf.keras.layers.Conv1DTranspose(9,  24*2+1, activation='relu',)#input_shape=input_shape[1:])
        
         
    def call(self, inputs):
        #LSTM파트
        x = self.block1_layer1(inputs)
        
        x = tf.nn.relu(x)
        x = self.block1_layer2(x)
        
        x = tf.nn.relu(x)
        x = self.block1_layer3(x)
       
        return x

# Eencoder()(x_train[0][:2,:,:]).shape
# Ddecoder()(Eencoder()(x_train[0][:2,:,:]))

class Autoencoder(tf.keras.Model): 
  def __init__(self,): 
    super(Autoencoder, self).__init__() 
    self.encoder = Eencoder() 
    self.decoder = Ddecoder() 
  
  def call(self, input): 
    code = self.encoder(input) 
    reconstructed = self.decoder(code) 
    return reconstructed

def loss(model, original): 
  reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original))) 
  return reconstruction_error

def train(loss, model, opt, original): 
  with tf.GradientTape() as tape: 
    gradients = tape.gradient(loss(model, original), model.trainable_variables) 
    gradient_variables = zip(gradients, model.trainable_variables) 
    opt.apply_gradients(gradient_variables)



In [ ]:
automodel = Autoencoder()
opt = tf.optimizers.Adam()
loss_fn = keras.losses.MeanSquaredError()
automodel.compile(optimizer=opt, loss=loss_fn)
automodel.fit(x_train[0], x_train[0], 
                 batch_size=1000, 
                 epochs=50,
                )

Epoch 1/50
1/1 [==============================] - 0s 430ms/step - loss: 0.3380
Epoch 2/50
1/1 [==============================] - 0s 22ms/step - loss: 0.3310
Epoch 3/50
1/1 [==============================] - 0s 21ms/step - loss: 0.3147
Epoch 4/50
1/1 [==============================] - 0s 22ms/step - loss: 0.2815
Epoch 5/50
1/1 [==============================] - 0s 22ms/step - loss: 0.2307
Epoch 6/50
1/1 [==============================] - 0s 20ms/step - loss: 0.2131
Epoch 7/50
1/1 [==============================] - 0s 22ms/step - loss: 0.2191
Epoch 8/50
1/1 [==============================] - 0s 21ms/step - loss: 0.1809
Epoch 9/50
1/1 [==============================] - 0s 20ms/step - loss: 0.1672
Epoch 10/50
1/1 [==============================] - 0s 21ms/step - loss: 0.1648
Epoch 11/50
1/1 [==============================] - 0s 22ms/step - loss: 0.1570
Epoch 12/50
1/1 [==============================] - 0s 20ms/step - loss: 0.1429
Epoch 13/50
1/1 [==============================] - 0s 22ms/s

In [ ]:
y_pred = automodel(x_test[0])
print("testset에 오토인코더 손실함수 테스트: ",loss_fn(x_test[0],y_pred))

testset에 오토인코더 손실함수 테스트:  tf.Tensor(0.04912294, shape=(), dtype=float32)


In [ ]:
automodel.encoder(x_train[0]).numpy().shape#.reshape(-1,36)

(100, 1, 36)

In [ ]:
class preprmodel(keras.Model):
    def __init__(self, imgmodel,autoencoder,name = None):
        super(preprmodel, self).__init__()
        self.imgmodel = imgmodel
        self.autoencoder = autoencoder

    def call(self, inputs):
        x1 = self.imgmodel(inputs[1])
        x2 = self.autoencoder.encoder(inputs[0]).numpy().reshape(-1,36)

        print(x1.shape,x2.shape)
        x = tf.concat([x1,x2],axis=1)
        
        return(x)
model_RESNET50 = ResNet50(weights='imagenet')
prepromodel = preprmodel(model_RESNET50,automodel)
x_prime_train = prepromodel(x_train)
x_prime_test = prepromodel(x_test)

(100, 1000) (100, 36)
(100, 1000) (100, 36)


In [ ]:
xgb = XGBClassifier(random_state=100, subsample= 0.7, colsample_bytree=0.7, scale_pos_weight=0.2)
xgb.fit(x_prime_train,y_train)

XGBClassifier(colsample_bytree=0.7, objective='multi:softprob',
              random_state=100, scale_pos_weight=0.2, subsample=0.7)

In [ ]:
y_pred=xgb.predict(x_prime_test)
# answer = np.array([label_description[label_decoder[int(val)]] for val in y_test])
# predss = np.array([label_description[label_decoder[int(val)]] for val in y_pred])

# new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
# new_crosstab

preds,시설포도_정상,시설포도노균병_중기,시설포도탄저병_초기,일소피해_말기,일소피해_초기,파프리카_정상,파프리카흰가루병_말기,파프리카흰가루병_중기,파프리카흰가루병_초기
answer,,,,,,,,,
시설포도_정상,39,0,0,0,0,0,0,0,0
시설포도노균병_중기,0,3,0,0,0,0,0,0,0
시설포도노균병_초기,0,0,0,0,0,0,0,1,0
시설포도탄저병_초기,0,0,2,0,0,0,0,0,0
일소피해_말기,0,0,0,1,0,0,0,0,0
일소피해_초기,0,0,0,0,1,0,0,0,0
파프리카_정상,0,0,0,0,0,34,0,0,0
파프리카잘록병_말기,1,0,0,0,0,0,0,0,0
파프리카흰가루병_말기,0,0,0,0,0,0,1,0,1


In [206]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[34  0  0  0  0  0  0  0  0  0  0]
 [ 0  6  3  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0 39  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  3  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  1]]
              precision    recall  f1-score   support

         0.0      1.000     1.000     1.000        34
         1.0      0.429     0.667     0.522         9
         2.0      0.000     0.000     0.000         7
         3.0      1.000     0.500     0.667         2
         6.0      0.000     0.000     0.000         1
        19.0      0.975     1.000     0.987        39
        20.0      1.000     1.000     1.000         2
        23.0      0.000     0.000     0.000         1
        24.0      1.000     1.000     1.000         3
        26.0      1.000     1.000     1.000         1
        

C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo